## Project Collaboration and Competition - Report

### DDPG and MADDPG Algorithms 

In this project, we use the **DDPG** algorithm (Deep Deterministic Policy Gradient) and the **MADDPG** algorithm,     
a wrapper for DDPG. MADDPG stands for **Multi-Agent DDPG**. DDPG is an algorithm which concurrently   learns    
a Q-function and a policy.  It uses off-policy data and the Bellman equation to learn the Q-function, 
and uses    
the Q-function to learn the policy. This dual mechanism is the  actor-critic method. The DDPG algorithm uses   
two additional mechanisms: _Replay Buffer_ and _Soft Updates_.  

In MADDPG, we train two separate agents, and the agents need to **collaborate** (like don’t let the   ball hit the ground)   
and **compete** (like gather as many points as possible). Just doing a simple extension of single 
agent RL    
by independently training the two agents does not work very well because the agents are independently updating    
their policies as learning progresses. And this causes the   environment to appear non-stationary from the viewpoint   
of any one agent. 

In MADDPG, _each agent’s critic is trained using the observations and actions_ from **both agents** , whereas   
each _agent’s actor is trained using just_ its **own observations**.  

In the finction _step()_ of the _class madppg_\__agent_, we collect all current info
 for **both agents**  into  the **common** variable    
_memory_ of the type  _ReplayBuffer_.  Then we get the random _sample_ from _memory_  into the variable _experiance_.   
This _experiance_   together with the current number of agent (0 or 1) go to the function _learn()_.   We get the corresponding    
agent (of type _ddpg_\__agent_):

      agent = self.agents[agent_number]

and _experiance_ is transferred to function _learn()_  of the _class ddpg_\__agent_.  There, the actor and the critic 
are handled by different ways.  







 ### Eight Neural Networks

In this prohect, there are 8 _neural networks_.  For the training, we 
create one _maddpg agent_. 

         maddpg = maddpg_agent()
         
In turn, _maddpg agent_ creates 2 _ddpg agents_: 
         
         self.agents = [ddpg_agent(state_size, action_size, i+1, random_seed=0) 
                  for i in range(num_agents)]    

Each of two agents (red and blue) create 4 neural networks:

        self.actor_local = Actor(state_size, action_size).to(device)
        self.actor_target = Actor(state_size, action_size).to(device)

        self.critic_local = Critic(state_size, action_size).to(device)
        self.critic_target = Critic(state_size, action_size).to(device)

Classes Actor and Critic are provided by **model.py**. The typical behavior of the actor 

        actor_target(state) -> next_actions
        actor_local(states) -> actions_pred
        
see function _learn()_ in maddpg agent. The typical behavior of the critic is as follows:

        critic_target(state, action) -> Q-value 
        -critic_local(states, actions_pred) -> actor_loss
        
see function _learn()_ in ddpg agent.        
        

### Architecture of the actor and critic networks

Both the actor and critic classes implement the neural network
with 3 fully-connected layers and 2 rectified nonlinear layers. These networks are realized in the framework
of package PyTorch. Such a network is used in Udacity model.py code for the Pendulum model using DDPG.
The number of neurons of the fully-connected layers are as follows:

for the actor:   
Layer fc1, number of neurons: state_size x fc1_units,   
Layer fc2, number of neurons: fc1_units x fc2_units,   
Layer fc3, number of neurons: fc2_units x action_size,   

for the critic:   
Layer fcs1, number of neurons: (state_size + action_size) x n_agents x fcs1_units,   
Layer fc2, number of neurons: (fcs1_units x fc2_units,   
Layer fc3, number of neurons: fc2_units x 1.   

Here, state_size = 24, action_size = 2.       
The input parameters fc1_units, fc2_units, fcs1_units are all taken = 64.   

### Hyperparameters

From **ddpg_agent.py** 

        GAMMA = 0.99                    # discount factor  
        TAU = 5e-2                      # for soft update of target parameters   
        LR_ACTOR = 5e-4                 # learning rate of the actor   
        LR_CRITIC = 5e-4                # learning rate of the critic  
        WEIGHT_DECAY = 0.0              # L2 weight decay   
        NOISE_AMPLIFICATION = 1         # exploration noise amplification  
        NOISE_AMPLIFICATION_DECAY = 1   # noise amplification decay

From **maddpg_agent.py**

        BUFFER_SIZE = int(1e6)          # replay buffer size   
        BATCH_SIZE = 512                # minibatch size   
        LEARNING_PERIOD = 2             # weight update frequency 
        
Note that parameters LEARNING_PERIOD is important. The corresponding code is in the function   _step()_.

     if len(self.memory) > BATCH_SIZE and timestep % LEARNING_PERIOD == 0: 
         for a_i, agent in enumerate(self.agents):
              experiences = self.memory.sample()
              self.learn(experiences, a_i)



### Training the Agent

On my local machine with GPU, the desired average reward **+0.5** was achieved in **1302** episodes in **18** minutes.
However, the training continued until **1700** episodes, where average reward was achived **+1.42** in **1 hour 18 minutes**.

### Full log

Episode: 20, Score: -0.0050, 	Average Score: -0.0050, Time: 00:00:00   
Episode: 40, Score: -0.0050, 	Average Score: -0.0050, Time: 00:00:02   
Episode: 60, Score: -0.0050, 	Average Score: -0.0050, Time: 00:00:07   
Episode: 80, Score: -0.0050, 	Average Score: -0.0050, Time: 00:00:12   
Episode: 100, Score: -0.0050, 	Average Score: -0.0050, Time: 00:00:18    
Episode: 120, Score: -0.0050, 	Average Score: -0.0040, Time: 00:00:23    
Episode: 140, Score: -0.0050, 	Average Score: -0.0025, Time: 00:00:30   
Episode: 160, Score: -0.0050, 	Average Score: -0.0020, Time: 00:00:36   
Episode: 180, Score: -0.0050, 	Average Score: -0.0015, Time: 00:00:43    
Episode: 200, Score: 0.0450, 	Average Score: 0.0040, Time: 00:00:53   
Episode: 220, Score: -0.0050, 	Average Score: 0.0070, Time: 00:01:03   
Episode: 240, Score: 0.0450, 	Average Score: 0.0065, Time: 00:01:09   
Episode: 260, Score: -0.0050, 	Average Score: 0.0085, Time: 00:01:17   
Episode: 280, Score: -0.0050, 	Average Score: 0.0080, Time: 00:01:22    
Episode: 300, Score: -0.0050, 	Average Score: 0.0045, Time: 00:01:30    
Episode: 320, Score: -0.0050, 	Average Score: 0.0035, Time: 00:01:38    
Episode: 340, Score: 0.0450, 	Average Score: 0.0045, Time: 00:01:46    
Episode: 360, Score: -0.0050, 	Average Score: 0.0030, Time: 00:01:52    
Episode: 380, Score: -0.0050, 	Average Score: 0.0030, Time: 00:01:58    
Episode: 400, Score: -0.0050, 	Average Score: 0.0010, Time: 00:02:04    
Episode: 420, Score: -0.0050, 	Average Score: -0.0005, Time: 00:02:11    
Episode: 440, Score: -0.0050, 	Average Score: 0.0010, Time: 00:02:20    
Episode: 460, Score: -0.0050, 	Average Score: 0.0015, Time: 00:02:27    
Episode: 480, Score: -0.0050, 	Average Score: 0.0035, Time: 00:02:35    
Episode: 500, Score: -0.0050, 	Average Score: 0.0085, Time: 00:02:43    
Episode: 520, Score: -0.0050, 	Average Score: 0.0075, Time: 00:02:49    
Episode: 540, Score: -0.0050, 	Average Score: 0.0065, Time: 00:02:57    
Episode: 560, Score: -0.0050, 	Average Score: 0.0050, Time: 00:03:03   
Episode: 580, Score: -0.0050, 	Average Score: 0.0050, Time: 00:03:10    
Episode: 600, Score: -0.0050, 	Average Score: 0.0030, Time: 00:03:17    
Episode: 620, Score: -0.0050, 	Average Score: 0.0050, Time: 00:03:25    
Episode: 640, Score: -0.0050, 	Average Score: 0.0110, Time: 00:03:38    
Episode: 660, Score: 0.0950, 	Average Score: 0.0130, Time: 00:03:46    
Episode: 680, Score: -0.0050, 	Average Score: 0.0210, Time: 00:03:59    
Episode: 700, Score: -0.0050, 	Average Score: 0.0185, Time: 00:04:05    
Episode: 720, Score: -0.0050, 	Average Score: 0.0160, Time: 00:04:10    
Episode: 740, Score: -0.0050, 	Average Score: 0.0075, Time: 00:04:16    
Episode: 760, Score: -0.0050, 	Average Score: 0.0065, Time: 00:04:23    
Episode: 780, Score: -0.0050, 	Average Score: 0.0045, Time: 00:04:34    
Episode: 800, Score: -0.0050, 	Average Score: 0.0080, Time: 00:04:43    
Episode: 820, Score: 0.0450, 	Average Score: 0.0110, Time: 00:04:51   
Episode: 840, Score: 0.1450, 	Average Score: 0.0180, Time: 00:05:07   
Episode: 860, Score: 0.0450, 	Average Score: 0.0305, Time: 00:05:28  
Episode: 880, Score: 0.0450, 	Average Score: 0.0315, Time: 00:05:42   
Episode: 900, Score: 0.0450, 	Average Score: 0.0415, Time: 00:06:00    
Episode: 920, Score: 0.0450, 	Average Score: 0.0510, Time: 00:06:17   
Episode: 940, Score: 0.0450, 	Average Score: 0.0545, Time: 00:06:34   
Episode: 960, Score: 0.1450, 	Average Score: 0.0535, Time: 00:06:51   
Episode: 980, Score: 0.0450, 	Average Score: 0.0610, Time: 00:07:12   
Episode: 1000, Score: 0.0450, 	Average Score: 0.0615, Time: 00:07:28  
Episode: 1020, Score: -0.0050, 	Average Score: 0.0550, Time: 00:07:38  
Episode: 1040, Score: 0.1450, 	Average Score: 0.0510, Time: 00:07:48   
Episode: 1060, Score: 0.0450, 	Average Score: 0.0570, Time: 00:08:08  
Episode: 1080, Score: 0.0450, 	Average Score: 0.0565, Time: 00:08:25  
Episode: 1100, Score: 0.0450, 	Average Score: 0.0670, Time: 00:08:49  
Episode: 1120, Score: 0.1450, 	Average Score: 0.0865, Time: 00:09:15  
Episode: 1140, Score: 0.0450, 	Average Score: 0.1115, Time: 00:09:46  
Episode: 1160, Score: 0.0450, 	Average Score: 0.1170, Time: 00:10:11  
Episode: 1180, Score: 0.2950, 	Average Score: 0.1250, Time: 00:10:36  
Episode: 1200, Score: -0.0050, 	Average Score: 0.1160, Time: 00:10:53  
Episode: 1220, Score: 2.3450, 	Average Score: 0.1350, Time: 00:11:39  
Episode: 1240, Score: 0.0450, 	Average Score: 0.1660, Time: 00:12:38  
Episode: 1260, Score: 0.1950, 	Average Score: 0.2350, Time: 00:14:01  
Episode: 1280, Score: 0.0450, 	Average Score: 0.3336, Time: 00:15:56  
Episode: 1300, Score: 0.0450, 	Average Score: 0.4592, Time: 00:18:07  
 
*** Environment solved in 1302 episodes!	Average Score: 0.51 ***   

Episode: 1320, Score: 0.0450, 	Average Score: 0.5434, Time: 00:20:07   
Episode: 1340, Score: 0.0450, 	Average Score: 0.4999, Time: 00:20:30   
Episode: 1360, Score: 0.0450, 	Average Score: 0.5736, Time: 00:22:51   
Episode: 1380, Score: 0.0450, 	Average Score: 0.4900, Time: 00:23:33   
Episode: 1400, Score: 0.0450, 	Average Score: 0.3955, Time: 00:24:17   
Episode: 1420, Score: 0.0450, 	Average Score: 0.3785, Time: 00:25:51   
Episode: 1440, Score: 2.6000, 	Average Score: 0.6059, Time: 00:29:11   
*** Episode 1450	Average Score: 0.55, Time: 00:29:18 ***   
 
Episode: 1460, Score: 1.8950, 	Average Score: 0.5722, Time: 00:31:11   
Episode: 1480, Score: 0.0450, 	Average Score: 0.6788, Time: 00:33:26   
Episode: 1500, Score: 0.7950, 	Average Score: 0.9267, Time: 00:37:45   
*** Episode 1500	Average Score: 0.93, Time: 00:37:45 ***   
 
Episode: 1520, Score: -0.0050, 	Average Score: 0.9882, Time: 00:40:17   
Episode: 1540, Score: 2.6000, 	Average Score: 0.9486, Time: 00:43:17   
*** Episode 1550	Average Score: 1.11, Time: 00:46:03 ***
 
Episode: 1560, Score: 2.6500, 	Average Score: 1.0685, Time: 00:47:36   
Episode: 1580, Score: 2.6000, 	Average Score: 1.1664, Time: 00:51:30   
Episode: 1600, Score: 0.0450, 	Average Score: 1.1409, Time: 00:55:57   
*** Episode 1600	Average Score: 1.14, Time: 00:55:57 ***  
 
Episode: 1620, Score: 1.4950, 	Average Score: 1.1379, Time: 00:58:45   
Episode: 1640, Score: 0.0950, 	Average Score: 1.1464, Time: 01:02:11   
*** Episode 1650	Average Score: 1.16, Time: 01:05:03 ***  
 
Episode: 1660, Score: 0.0450, 	Average Score: 1.2741, Time: 01:08:14   
Episode: 1680, Score: 2.6000, 	Average Score: 1.3547, Time: 01:13:11    
Episode: 1700, Score: 0.0950, 	Average Score: 1.4214, Time: 01:18:16    
*** Episode 1700	Average Score: 1.42, Time: 01:18:16 ***  

### Future ideas

1. Check different values for hyperparameters such as LEARNING_PERIOD, and neural network parameters fc1_units, fc2_units, etc.
2. How does the addition of new nonlinear layers in the used neural networks affect the robustness of the algorithm.
3. It would be interesting to train agents using [MAPPO](https://github.com/kotogasy/unity-ml-tennis) and compare them with MADDPG. 